In [18]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf

import tensorflow as tf
import tensorflow as tf
import keras
from keras import layers, Sequential

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Load the dataset
df = pd.read_csv('/Users/hcaap/Desktop/Kod/Siemens/Clusterings/test/multilabel_data.csv')

df['Labels'] = df['Labels'].apply(lambda x: x.split(','))

df.head()



,Input Text,Labels
0,Blade tip,[Blade Tip Cluster]
1,Tip,[Blade Tip Cluster]
2,Blade tips,[Blade Tip Cluster]
3,Tip of blade,[Blade Tip Cluster]
4,Leading edge tip,"[Blade Tip Cluster, Leading Edge Cluster]"


In [23]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import layers, Sequential
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Define the mapping of keywords to clusters
keyword_to_clusters = {
    "Leading edge": ["Leading Edge Cluster"],
    "Tip": ["Blade Tip Cluster"],
    "Suction side": ["Suction Side Cluster"],
    "Pressure side": ["Pressure Side Cluster"],
    "Trailing edge": ["Trailing Edge Cluster"],
    "Leading edge tip": ["Blade Tip Cluster", "Leading Edge Cluster"],
    # Add more mappings as needed
}

# Function to assign labels based on keywords
def assign_labels(text, keyword_to_clusters):
    labels = []
    for keyword, clusters in keyword_to_clusters.items():
        if keyword.lower() in text.lower():
            labels.extend(clusters)
    return list(set(labels))  # Remove duplicates

# Load the existing dataset
df = pd.read_csv('/Users/hcaap/Desktop/Kod/Siemens/Clusterings/test/multilabel_data.csv')
df['Labels'] = df['Labels'].apply(lambda x: x.split(','))

# Sample new data
new_data = [
    ("Leading edge tip",),
    ("Trailing edge cooling holes",),
    ("Suction side of airfoil",),
    # Add more data tuples as needed
]

# Process the new data
processed_data = []
for text_tuple in new_data:
    text = text_tuple[0]
    labels = assign_labels(text, keyword_to_clusters)
    processed_data.append((text, labels))

# Convert to DataFrame
new_df = pd.DataFrame(processed_data, columns=["Input Text", "Labels"])

new_df.head()


,Input Text,Labels
0,Leading edge tip,"[Blade Tip Cluster, Leading Edge Cluster]"
1,Trailing edge cooling holes,[Trailing Edge Cluster]
2,Suction side of airfoil,[Suction Side Cluster]


In [19]:

# MultiLabelBinarizer for encoding labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Labels'])

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['Input Text']).toarray()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.77764348, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [20]:
# Define the model
model = Sequential()
# Define the model
model = keras.Sequential()
model.add(layers.Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(y_train.shape[1], activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/Users/hcaap/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

print(X_test)
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes, average='weighted')
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.0379 - loss: 0.6930 - val_accuracy: 0.0556 - val_loss: 0.6824
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0301 - loss: 0.6781 - val_accuracy: 0.0556 - val_loss: 0.6699
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0112 - loss: 0.6652 - val_accuracy: 0.0556 - val_loss: 0.6560
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0524 - loss: 0.6505 - val_accuracy: 0.0000e+00 - val_loss: 0.6401
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0491 - loss: 0.6322 - val_accuracy: 0.0000e+00 - val_loss: 0.6215
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0602 - loss: 0.6151 - val_accuracy: 0.0000e+00 - val_loss: 0.5997
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0184 - loss: 0.5900 - val_accuracy: 0.0000e+00 - val_loss: 0.5745
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0602 - loss: 0.5610 - val_accuracy: 0.0000e+00 - val

/Users/hcaap/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hcaap/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/hcaap/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
# Define the new input text
new_input_text = "Leading edge platform"

# Preprocess the input text
new_input_vector = tfidf.transform([new_input_text]).toarray()

# Predict with the model
predicted_probabilities = model.predict(new_input_vector)

# Convert probabilities to class labels
threshold = 0.1
predicted_classes = (predicted_probabilities > threshold).astype(int)

# Decode the predicted classes
predicted_labels = mlb.inverse_transform(predicted_classes)

# Output the predicted labels
print(f"Input Text: {new_input_text}")
print(f"Predicted Labels: {predicted_labels}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Input Text: Leading edge platform
Predicted Labels: [('Airfoil Cluster', 'Blade Body/Profile Cluster', 'Blade Tip Cluster', 'Concave Side Cluster', 'Convex Side Cluster', 'Cooling Holes Cluster', 'Cooling Passages Cluster', 'General Edges Cluster', 'Leading Edge Cluster', 'Miscellaneous Cluster', 'Platform Cluster: General Platform', 'Platform Cluster: Inner Platform', 'Platform Cluster: Outer Platform', 'Platform Cluster: Rear Platform', 'Pressure Side Cluster', 'Root/Attachment Cluster', 'Stage-related Cluster', 'Suction Side Cluster', 'Surface Cluster', 'Trailing Edge Cluster', 'Unspecified/Unknown Cluster')]


In [24]:
def assign_labels(text, clusters):
    labels = []
    for cluster_name, keywords in clusters.items():
        for keyword in keywords:
            if keyword.lower() in text.lower():
                labels.append(cluster_name)
                break
    return list(set(labels))  # Remove duplicates

# Define clusters with keywords
clusters = {
    "Blade Tip Cluster": ["Blade tip", "Tip", "Blade tips", "Tip of blade", "Tip area", "Tip suction side", "Tip pressure side", "Near blade tips", "Top of blade tip", "Leading edge tip"],
    "Leading Edge Cluster": ["Leading edge", "Leading edge cooling holes", "Leading edge platform", "Leading edge side", "Leading edge tip area", "Leading edge base platform", "Leading edge outer part", "Leading edge airfoil", "Leading edge near tip area", "Around leading edge"],
    "Suction Side Cluster": ["Suction side", "Suction side of airfoil", "Suction side tip", "Suction side platform", "Suction side trailing edge", "Suction side of blade", "Suction side of trailing edge", "On the suction side of the blade"],
    "Trailing Edge Cluster": ["Trailing edge", "Trailing edge platform", "Trailing edge cooling holes", "Trailing edge rear platform", "Trailing edge tip", "Trailing edge side", "Trailing edge of platform", "Trailing edge around platform"],
    "Pressure Side Cluster": ["Pressure side", "Pressure side of airfoil", "Pressure side tip", "Pressure tip", "On the pressure side"],
    "Platform Cluster": ["Rear platform", "Platform", "Inner platform", "Outer platform", "Blade platform", "Top platform", "Base platform", "Front platform", "Mid platform", "Rear outer platform", "Inner platform leading edge", "Inner platform trailing edge", "Outer platform leading edge", "Outer platform trailing edge"],
    "Cooling Holes Cluster": ["Cooling holes", "Leading edge cooling holes", "Trailing edge cooling holes", "Cooling air holes", "Cooling entrance orifice plate", "Cooling air passages"],
    "Airfoil Cluster": ["Airfoil", "Leading edge of airfoil", "Suction side of airfoil", "Airfoil surface", "On airfoil"],
    "Surface Cluster": ["Surface", "Blade surface", "Gas washed surfaces", "On blade tip", "On platform", "Blade roots", "On airfoil"],
    "Root/Attachment Cluster": ["Blade root", "Blade roots", "Foot of blade", "Blade foot", "Root", "Blade attachments", "Blade attachment slots", "Foot at leading edge"],
    "Edge Cluster": ["Leading edge", "Trailing edge", "Inlet edge", "Front edge"],
    "Blade Body/Profile Cluster": ["Blade", "Blades", "Blade top", "Blade tops", "Top of blade", "Top of blades", "Top section", "Blade body", "Blade edge"],
    "Unspecified/Unknown Cluster": ["Unspecified", "Not specified", "Unknown"],
    "Stage-related Cluster": ["Stage #1", "Stage #2", "Stage #3", "Stage #4", "Stage #5", "Stage #6", "Stage #7", "Stage #8", "Stage #9", "Stage #10", "Stage #11", "Stage #12", "Stage #13", "Stage #14", "Stage #15"],
    "Miscellaneous Cluster": ["Rear part", "Camber side"]
}

# Example usage
example_text = "Leading edge tip"
labels = assign_labels(example_text, clusters)
print(f"Input Text: {example_text}")
print(f"Predicted Labels: {labels}")

Input Text: Leading edge tip
Predicted Labels: ['Blade Tip Cluster', 'Edge Cluster', 'Leading Edge Cluster']


In [56]:
def assign_labels(text, clusters):
    labels = []
    for main_cluster, sub_clusters in clusters.items():
        for sub_cluster_name, keywords in sub_clusters.items():
            for keyword in keywords:
                if keyword.lower() in text.lower():
                    labels.append(f"{main_cluster} -> {sub_cluster_name}")
                    break
    return list(set(labels))  # Remove duplicates

# Define clusters with sub-clusters and keywords
clusters = {
    "Edge Cluster": {
        "Leading Edge Sub-Cluster": ["Leading edge", "Leading edge cooling holes", "Leading edge platform", "Leading edge side", "Leading edge tip area", "Leading edge base platform", "Leading edge outer part", "Leading edge airfoil", "Leading edge near tip area", "Around leading edge"],
        "Trailing Edge Sub-Cluster": ["Trailing edge", "Trailing edge platform", "Trailing edge cooling holes", "Trailing edge rear platform", "Trailing edge tip", "Trailing edge side", "Trailing edge of platform", "Trailing edge around platform"],
        "Inlet Edge Sub-Cluster": ["Inlet edge", "Front edge"]
    },
    "Surface Cluster": {
        "Suction Side Sub-Cluster": ["Suction side", "Suction side of airfoil", "Suction side tip", "Suction side platform", "Suction side trailing edge", "Suction side of blade", "Suction side of trailing edge", "On the suction side of the blade"],
        "Pressure Side Sub-Cluster": ["Pressure side", "Pressure side of airfoil", "Pressure side tip", "Pressure tip", "On the pressure side"],
        "Concave Side Sub-Cluster": ["Concave side"],
        "Convex Side Sub-Cluster": ["Convex side"]
    },
    "Blade Body/Profile Cluster": {
        "Blade Tip Sub-Cluster": ["Blade tip", "Tip", "Blade tips", "Tip of blade", "Tip area", "Tip suction side", "Tip pressure side", "Near blade tips", "Top of blade tip", "Leading edge tip"],
        "General Blade Sub-Cluster": ["Blade", "Blades", "Blade top", "Blade tops", "Top of blade", "Top of blades", "Top section", "Blade body", "Blade edge"]
    },
    "Platform Cluster": {
        "Inner Platform Sub-Cluster": ["Inner platform", "Inner platform leading edge", "Inner platform trailing edge"],
        "Outer Platform Sub-Cluster": ["Outer platform", "Outer platform leading edge", "Outer platform trailing edge"],
        "General Platform Sub-Cluster": ["Rear platform", "Platform", "Blade platform", "Top platform", "Base platform", "Front platform", "Mid platform", "Rear outer platform"]
    },
    "Cooling Holes/Passages Cluster": {
        "Cooling Holes Sub-Cluster": ["Cooling holes", "Leading edge cooling holes", "Trailing edge cooling holes", "Cooling air holes", "Cooling entrance orifice plate", "Cooling air passages"]
    },
    "Airfoil Cluster": {
        "Airfoil": ["Airfoil", "Leading edge of airfoil", "Suction side of airfoil", "Airfoil surface", "On airfoil"]
    },
    "Root/Attachment Cluster": {
        "Root/Attachment": ["Blade root", "Blade roots", "Foot of blade", "Blade foot", "Root", "Blade attachments", "Blade attachment slots", "Foot at leading edge"]
    },
    "Unspecified/Unknown Cluster": {
        "Unspecified/Unknown": ["Unspecified", "Not specified", "Unknown"]
    },
    "Stage-related Cluster": {
        "Stage-related": ["Stage #1", "Stage #2", "Stage #3", "Stage #4", "Stage #5", "Stage #6", "Stage #7", "Stage #8", "Stage #9", "Stage #10", "Stage #11", "Stage #12", "Stage #13", "Stage #14", "Stage #15"]
    },
    "Miscellaneous Cluster": {
        "Miscellaneous": ["Rear part", "Camber side"]
    }
}

# Example usage
example_text = "Leading edge tip"
labels = assign_labels(example_text, clusters)
print(f"Input Text: {example_text}")
print(f"Predicted Labels: {labels}")

Input Text: Leading edge tip
Predicted Labels: ['Edge Cluster -> Leading Edge Sub-Cluster', 'Blade Body/Profile Cluster -> Blade Tip Sub-Cluster']


In [27]:
# Sample new data
new_data = [
    ("Leading edge tip",),
    ("Trailing edge cooling holes",),
    ("Suction side of airfoil",),
    # Add more data tuples as needed
]

# Process the new data
processed_data = []
for text_tuple in new_data:
    text = text_tuple[0]
    labels = assign_labels(text, clusters)
    processed_data.append((text, labels))

# Convert to DataFrame
new_df = pd.DataFrame(processed_data, columns=["Input Text", "Labels"])

# Combine DataFrames
combined_df = pd.concat([df, new_df], ignore_index=True)

combined_df.head()

,Input Text,Labels
0,Blade tip,[Blade Tip Cluster]
1,Tip,[Blade Tip Cluster]
2,Blade tips,[Blade Tip Cluster]
3,Tip of blade,[Blade Tip Cluster]
4,Leading edge tip,"[Blade Tip Cluster, Leading Edge Cluster]"


In [72]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow as tf
import keras
from keras import layers, Sequential
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

clusters_with_heads = {
    "Edge Cluster": {
        "head_keywords": ["edge"],
        "sub_clusters": {
            "Leading Edge Sub-Cluster": ["Leading edge", "Leading edge cooling holes", "Leading edge platform", "Leading edge side", "Leading edge tip area", "Leading edge base platform", "Leading edge outer part", "Leading edge airfoil", "Leading edge near tip area", "Around leading edge"],
            "Trailing Edge Sub-Cluster": ["Trailing edge", "Trailing edge platform", "Trailing edge cooling holes", "Trailing edge rear platform", "Trailing edge tip", "Trailing edge side", "Trailing edge of platform", "Trailing edge around platform"],
            "Inlet Edge Sub-Cluster": ["Inlet edge", "Front edge"]
        }
    },
    "Side Cluster": {
        "head_keywords": ["side"],
        "sub_clusters": {
            "Suction Side Sub-Cluster": ["Suction side", "Suction side of airfoil", "Suction side tip", "Suction side platform", "Suction side trailing edge", "Suction side of blade", "Suction side of trailing edge", "On the suction side of the blade"],
            "Pressure Side Sub-Cluster": ["Pressure side", "Pressure side of airfoil", "Pressure side tip", "Pressure tip", "On the pressure side"],
            "Concave Side Sub-Cluster": ["Concave side"],
            "Convex Side Sub-Cluster": ["Convex side"]
        }
    },
    "Blade Body/Profile Cluster": {
        "head_keywords": ["blade"],
        "sub_clusters": {
            "Blade Tip Sub-Cluster": ["Blade tip", "Tip", "Blade tips", "Blade top", "Blade tops", "Top of blade", "Top of blades", "Top section",  "Tip of blade", "Tip area", "Tip suction side", "Tip pressure side", "Near blade tips", "Top of blade tip", "Leading edge tip"],
            "General Blade Sub-Cluster": ["Blade", "Blades", "Blade body", "Blade edge"]
        }
    },
    "Platform Cluster": {
        "head_keywords": ["platform"],
        "sub_clusters": {
            "Inner Platform Sub-Cluster": ["Inner", "Inner platform", "Inner platform leading edge", "Inner platform trailing edge"],
            "Outer Platform Sub-Cluster": ["Outer platform", "Outer platform leading edge", "Outer platform trailing edge"],
            "General Platform Sub-Cluster": ["Rear platform", "Platform", "Blade platform", "Top platform", "Base platform", "Front platform", "Mid platform", "Rear outer platform"]
        }
    },
    "Cooling Holes/Passages Cluster": {
        "head_keywords": ["cooling", "holes", "passages"],
        "sub_clusters": {
            "Cooling Holes Sub-Cluster": ["Cooling holes", "Leading edge cooling holes", "Trailing edge cooling holes", "Cooling air holes", "Cooling entrance orifice plate", "Cooling air passages"]
        }
    },
    "Airfoil Cluster": {
        "head_keywords": ["airfoil"],
        "sub_clusters": {
            "Airfoil": ["Airfoil", "Leading edge of airfoil", "Suction side of airfoil", "Airfoil surface", "On airfoil"]
        }
    },
    "Root/Attachment Cluster": {
        "head_keywords": ["root", "attachment"],
        "sub_clusters": {
            "Root/Attachment": ["Blade root", "Blade roots", "Foot of blade", "Blade foot", "Root", "Blade attachments", "Blade attachment slots", "Foot at leading edge"]
        }
    },
    "Unspecified/Unknown Cluster": {
        "head_keywords": ["unspecified", "unknown"],
        "sub_clusters": {
            "Unspecified/Unknown": ["Unspecified", "Not specified", "Unknown"]
        }
    },
    "Stage-related Cluster": {
        "head_keywords": ["stage"],
        "sub_clusters": {
            "Stage-related": ["Stage #1", "Stage #2", "Stage #3", "Stage #4", "Stage #5", "Stage #6", "Stage #7", "Stage #8", "Stage #9", "Stage #10", "Stage #11", "Stage #12", "Stage #13", "Stage #14", "Stage #15"]
        }
    },
    "Miscellaneous Cluster": {
        "head_keywords": ["miscellaneous"],
        "sub_clusters": {
            "Miscellaneous": ["Rear part", "Camber side"]
        }
    }
}

def assign_labels(text, clusters):
    labels = []

    # Preprocess text to lower case for case-insensitive matching
    text = text.lower()
    words_in_text = set(text.split())

    for main_cluster, cluster_data in clusters.items():
        head_keywords = cluster_data["head_keywords"]
        sub_clusters = cluster_data["sub_clusters"]

        # Check for head keywords in the input text
        if any(keyword.lower() in words_in_text for keyword in head_keywords):
            print(f"Head keywords found in text: {head_keywords}")
            # Remove head keywords from the input text for sub-cluster matching
            filtered_words_in_text = words_in_text - set(head_keywords)
            print("remaining words: ", filtered_words_in_text)
            best_match = None
            best_match_count = 0
            
            # Check for specific matches and count overlapping keywords
            for sub_cluster_name, keywords in sub_clusters.items():
                keyword_set = set()
                for keyword in keywords:
                    keyword_set.update(keyword.lower().split())
                intersection_count = len(filtered_words_in_text & keyword_set)
                print(f"Checking sub-cluster: {sub_cluster_name}")
                print(f"Keywords in sub-cluster: {keyword_set}")
                intersection_count = len(filtered_words_in_text & keyword_set)
                print(f"Intersection count: {intersection_count}")

                # Check for exact match for whole keywords
                if any(keyword.lower() in filtered_words_in_text for keyword in keywords):
                    labels.append(f"{main_cluster} -> {sub_cluster_name}")
                    best_match = None  # Reset best match since we found an exact match
                    break

                # If no exact match is found, find the best match based on overlapping keywords
                if intersection_count > best_match_count:
                    best_match = sub_cluster_name
                    best_match_count = intersection_count

            # If no specific match found, assign the best match based on overlapping keywords
            if best_match:
                labels.append(f"{main_cluster} -> {best_match}")

    return list(set(labels))  # Remove duplicates

print("list : " , assign_labels("Leading edge cooling holes", clusters_with_heads))

# # Load the dataset
df = pd.read_csv('/Users/hcaap/Desktop/Kod/Siemens/Clusterings/test/multilabel_data.csv')

# # Assign labels to existing data
df['Labels'] = df['Input Text'].apply(lambda text: assign_labels(text, clusters_with_heads))

df.head()




Head keywords found in text: ['edge']
remaining words:  {'cooling', 'leading', 'holes'}
Checking sub-cluster: Leading Edge Sub-Cluster
Keywords in sub-cluster: {'cooling', 'tip', 'base', 'airfoil', 'holes', 'edge', 'part', 'outer', 'near', 'leading', 'platform', 'around', 'side', 'area'}
Intersection count: 3
Checking sub-cluster: Trailing Edge Sub-Cluster
Keywords in sub-cluster: {'cooling', 'trailing', 'tip', 'holes', 'edge', 'platform', 'around', 'side', 'of', 'rear'}
Intersection count: 2
Checking sub-cluster: Inlet Edge Sub-Cluster
Keywords in sub-cluster: {'inlet', 'front', 'edge'}
Intersection count: 0
Head keywords found in text: ['cooling', 'holes', 'passages']
remaining words:  {'edge', 'leading'}
Checking sub-cluster: Cooling Holes Sub-Cluster
Keywords in sub-cluster: {'cooling', 'trailing', 'holes', 'entrance', 'edge', 'air', 'plate', 'passages', 'leading', 'orifice'}
Intersection count: 2
list :  ['Cooling Holes/Passages Cluster -> Cooling Holes Sub-Cluster', 'Edge Cluster

,Input Text,Labels
0,Blade tip,[Blade Body/Profile Cluster -> Blade Tip Sub-C...
1,Tip,[]
2,Blade tips,[Blade Body/Profile Cluster -> Blade Tip Sub-C...
3,Tip of blade,[Blade Body/Profile Cluster -> Blade Tip Sub-C...
4,Leading edge tip,[Edge Cluster -> Leading Edge Sub-Cluster]
